In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline



In [2]:
# Load the dataset
df = pd.read_csv('cleaned_data.csv')



In [3]:
# Define features and target
X = df.drop(['delay_rrt'], axis=1)
y = df['delay_rrt']



In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [5]:
# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns



In [6]:
# Create preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_columns),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_columns)
])



In [7]:
# Create feature selector using Random Forest
feature_selector = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42),
    max_features=20
)



In [8]:
# Create pipeline with SMOTE for handling class imbalance
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', feature_selector),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42, class_weight='balanced'))
])



In [9]:
# Define broader hyperparameter space
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear', 'saga'],
    'classifier__max_iter': [1000]
}



In [10]:
# Perform cross-validation with multiple metrics
from sklearn.model_selection import GridSearchCV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring={
        'auc': 'roc_auc',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1'
    },
    refit='auc'
)



In [11]:
# Fit the model
grid_search.fit(X_train, y_train)



C:\Users\Harshvardhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Harshvardhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Harshvardhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['weight', 'aki_stage', 'gender', 'admission_age', 'hematocrit_min',
       'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'platelets_min',
       'platelets_max', 'wbc_min', 'wbc_max', 'anion...
                                       ('smote', SMOTE(random_state=42)),
                                       ('classifier',
                                        LogisticRegression(class_weight='balanced',
                                                           random_state=42))]),
             param_grid={'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'classifier__max_iter': [1000],
                         'classifier__penalty': ['l1', 'l2'],
                         'classifier__solver': ['liblinear', 'saga']},
             refit='auc',
             scoring={'auc': 'roc_auc', 'f1': 'f1', 'precision': 'precision',
                      'recall': 'recall'})

In [12]:
# Get best model
best_model = grid_search.best_estimator_



In [13]:
# Function to evaluate model
from sklearn.metrics import classification_report, confusion_matrix
def evaluate_model(model, X, y, dataset_name=""):
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    
    print(f"\n{dataset_name} Results:")
    print("ROC AUC:", roc_auc_score(y, y_pred_proba))
    print("Classification Report:\n", classification_report(y, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
    
    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y, y_pred_proba)
    print("Average Precision Score:", average_precision_score(y, y_pred_proba))



In [14]:
# Evaluate on both train and test sets
evaluate_model(best_model, X_train, y_train, "Training")
evaluate_model(best_model, X_test, y_test, "Test")




Training Results:
ROC AUC: 0.7738249507081298
Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.65      0.45       263
           1       0.90      0.72      0.80      1159

    accuracy                           0.71      1422
   macro avg       0.63      0.69      0.63      1422
weighted avg       0.80      0.71      0.74      1422

Confusion Matrix:
 [[172  91]
 [322 837]]
Average Precision Score: 0.933176535508764

Test Results:
ROC AUC: 0.6927899686520377
Classification Report:
               precision    recall  f1-score   support

           0       0.32      0.55      0.40        66
           1       0.88      0.73      0.80       290

    accuracy                           0.70       356
   macro avg       0.60      0.64      0.60       356
weighted avg       0.77      0.70      0.73       356

Confusion Matrix:
 [[ 36  30]
 [ 77 213]]
Average Precision Score: 0.9005162165481735


In [15]:
# Get feature importance
def get_feature_importance(model, feature_names):
    # Get the logistic regression coefficients
    if hasattr(model['classifier'], 'coef_'):
        importance = np.abs(model['classifier'].coef_[0])
        selected_features = model['feature_selector'].get_support()
        
        # Get transformed feature names
        transformed_features = []
        numeric_features = numeric_columns.tolist()
        
        # Add selected features only
        feature_importance_dict = {}
        idx = 0
        for i, is_selected in enumerate(selected_features):
            if is_selected:
                feature_importance_dict[feature_names[i]] = importance[idx]
                idx += 1
        
        return sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)



In [16]:

# Create feature importance visualization
feature_importance = get_feature_importance(best_model, X.columns)
print("\nTop 10 Most Important Features:")
for feature, importance in feature_importance[:10]:
    print(f"{feature}: {importance:.4f}")


Top 10 Most Important Features:
aniongap_min: 0.6808
creatinine_min: 0.6251
resp_rate_mean: 0.3031
pt_max: 0.2746
potassium_min: 0.2704
inr_max: 0.2677
chloride_max: 0.2505
aniongap_max: 0.1791
wbc_max: 0.1743
calcium_min: 0.1703
